# **Першопочаткове налаштування**

In [57]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

if tf.config.list_physical_devices('GPU'):
  print("TensorFlow IS using the GPU")
else:
  print("TensorFlow IS NOT using the GPU")

TensorFlow IS NOT using the GPU


Налаштування було успішним, у ході експериментів використовуватиметься GPU.

Перейдемо до реалізації практичного завдання лабораторної роботи, завантаживши класичний датасет для задачі класифікації зображень Fashion MNIST, що складається з зображень розміром 28*28 пікселів у сірому діапазоні кольорів.

Так як цей датасет вбудований в Keras, завантажимо його одразу:

In [58]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Завантажені зображення є, по суті, масивами пікселів, значення кожного з пікселів варіюється від 0 до 255, включно, а значення кожного з лейблів у масивах лейблів варіюються від 0 до 9 включно, ці значення є фактичними класами, до яких нейромережа приписуватиме зображення.

Так як ми хочемо отримати відносно точні результати у наслідку тренування нейромережі, одразу нормалізуємо формат даних таким чином, аби кожне значення використовуваних нейромережею масивів відповідало діапазону [0; 1].

Зробити це можна, поділивши значення масивів пікселів на 255, адже це є найбільшим значенням у масивах:

In [59]:
train_images = train_images / 255.0
test_images = test_images / 255.0

# **Побудова початкової моделі**

Побудуємо архітектуру простої секвентальної нейромережі (Sequential API):

In [60]:
model = keras.Sequential(
    [
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
    ],
    name='TDP1'
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Дана модель матиме три слої:

1.  Вхідний слой Flatten, за рахунок пре-процесингу якого двомірні матриці пікселів перегортаються в одномірні масиви чисел, з якими зможе працювати нейромережа;
2.  Прихований слой Dense, який є основним обчислювальним слоєм з 128 нейронами, активаційна функція relu використовується через свою поширеність у простих задачах класифікації зображень;
3.  Вихідний слой Dense, має 10 нейронів для кожного з класів зображень ([0; 9]), активаційна функція softmax використовується для автоматичної конвертації вихідних даних у відсоткові значення для кожного з 10 класів.



# **Компіляція початкової моделі**

Проведемо компіляцію моделі та виведемо її архітектуру:

In [61]:
model.compile(
    optimizer='adam',
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

У кожної мережі є три основні компіляційні параметри: функція оптимізації (optimizer), функція втрати (loss) і параметр виведення (metrics). Розглянемо вибір кожного з них:



*   Optimizer: було обрано Adam, адже ця функція є найбільш поширеною для задач класифікації;
*   Loss: було обрано Sparse Categorical Crossentropy, адже ця функція є найбільш поширеною для задач класифікації;
*   Metrics: було обрано accuracy, адже по ходу тестування ми бажаємо бачити саме точність моделі нейромережі.



Виведемо підсумки зкомпільованої моделі нейромережі:

In [62]:
model.summary()

Model: "TDP1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_16 (Flatten)            │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101,770 (397.54 KB)

 Trainable params: 101,770 (397.54 KB)

 Non-trainable params: 0 (0.00 B)

Як можемо побачити, у створеній моделі:


1.   На першому слої: 784 (28*28) вхідних параметри, і 0 вихідних, адже слой є виключно вхідним;
2.   На другому слої: 128 вхідних параметрів (за кількістю нейронів), і 100480 (784 * 128 + 128 зміщень за замовчуванням) вихідних параметрів;
3. На третьому слої: 10 вхідних параметрів (за кількістю нейронів), і 1290 (128 * 10 + 10 зміщень за замовчуванням) вихідних параметрів;



# **Тренування початкової моделі**

Виконаємо першопочаткове тренування моделі з 10 епохами, і виділенням 20% тренувальних даних (12000 з 60000) на валідацію, аби розміри даних для валідації (12000) та тестування (10000) були приблизно однаковими:

In [63]:
history = model.fit(train_images, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/backend/tensorflow/nn.py:717: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1500/1500 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.7800 - loss: 0.6463 - val_accuracy: 0.8558 - val_loss: 0.4106
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8589 - loss: 0.3990 - val_accuracy: 0.8640 - val_loss: 0.3776
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.8711 - loss: 0.3529 - val_accuracy: 0.8761 - val_loss: 0.3531
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8820 - loss: 0.3208 - val_accuracy: 0.8676 - val_loss: 0.3575
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.8874 - loss: 0.3053 - val_accuracy: 0.8845 - val_loss: 0.3259
Epoch 6/10
 205/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8937 - loss: 0.2794

KeyboardInterrupt: 

Тепер, виконаємо аналіз точності моделі на тестових даних:

In [ ]:
print("--- Перевіряємо точність моделі на тестових даних ---")
test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=2)
print("Тестова точність:", test_accuracy)
print("Тестова помилка:", test_loss)

Виведемо на графіку результати тестувань, що покажуть ефективність моделі:

In [ ]:
def display_results(history, model=None, title=None):
  name = (
        title
        or (getattr(model, 'name', None))
        or (getattr(getattr(history, 'model', None), 'name', None))
        or 'Модель'
      )

  history_dictionary = history.history
  accuracy = history_dictionary['accuracy']
  val_accuracy = history_dictionary['val_accuracy']
  loss = history_dictionary['loss']
  val_loss = history_dictionary['val_loss']
  epochs_range = range(1, len(accuracy) + 1)

  fig, axes = plt.subplots(1, 2, figsize=(12, 5))
  fig.suptitle(f'Модель: {name}', fontsize=14, y=1.03)

  plt.subplot(1, 2, 1)
  plt.plot(epochs_range, accuracy, 'o-', color='#1f77b4', label='Точність тренування')
  plt.plot(epochs_range, val_accuracy, '-', color='#ff7f0e', label='Точність валідації')
  plt.title('Точність тренування і валідації')
  plt.xlabel('Епохи')
  plt.ylabel('Точність')
  plt.ylim(0.5, 1.0)
  plt.legend()
  plt.grid(alpha=0.2)

  plt.subplot(1, 2, 2)
  plt.plot(epochs_range, loss, 'o-', color='#1f77b4', label='Помилка тренування')
  plt.plot(epochs_range, val_loss, '-', color='#ff7f0e', label='Помилка валідації')
  plt.title('Помилка тренування і валідації')
  plt.xlabel('Епохи')
  plt.ylabel('Помилка')
  plt.ylim(0.0, 0.5)
  plt.legend()
  plt.grid(alpha=0.2)

  fig.tight_layout(rect=[0, 0, 1, 0.95])
  plt.show()


In [ ]:
display_results(history)

Як можемо побачити, приблизно після 4 епох зменшується точність валідації даних, непропорційно тренувальним даним.

Спробуємо різні оптимізації, аби побачити їхній ефект на точності моделі, але спочатку збережемо першопочаткову модель:

In [ ]:
model.save('tdp1_model_v1.keras')
print('\nЗбережено модель tdp1_model_v1')

# **Оптимізація місткості моделі**

Спочатку, аби запобігти перенавчання у будь-якому майбутньому сценаріЇ, застосуємо метод ранньої зупинки зі значенням толерантності 10:

In [ ]:
early_stopping_callback = keras.callbacks.EarlyStopping(
    monitor='val_xent',
    patience=10,
    min_delta=1e-3,
    restore_best_weights=True,
    mode='min'
)

Застосуємо також поступове зниження швидкості навчання:

In [ ]:
lr_reduction_callback = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=4,
    cooldown=2,
    min_delta=1e-3,
    min_lr=1e-6,
    verbose=1,
    mode='min'
)

Так як тестування виконуватимуться на відносно невеликому та збалансованому датасеті, звичайне використання системи збереження контрольних точок під час тренування не є обов'язковим.

Натомість, застосуємо цю систему для збереження найкращої моделі під час кожного індивідуального тренування, таким чином, навіть якщо тренування буде перерване, існуватиме модель, з якої можна буде відновити тренування, і наприкінці можна буде порівняти результати такої моделі з фактичними фінальними:

In [ ]:
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath='best_model_checkpoint.keras',
    monitor='val_xent',
    save_best_only=True,
    mode='min',
    verbose=1
)

Протестуємо раніше створену модель з новими гіперпараметрами, але спочатку визначимо загальні методи:

In [ ]:
def get_optimizer(lr=1e-3):
  return keras.optimizers.Adam(learning_rate=lr)

def compile_and_fit(model, name=None, optimizer=None, max_epochs=10000, batch_size=None):
  if optimizer is None:
    optimizer = get_optimizer()

  model.compile(
    optimizer=optimizer,
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=[
        'accuracy',
        keras.losses.SparseCategoricalCrossentropy(name='xent')
            ]
    )

  model.summary()

  callbacks = [early_stopping_callback, lr_reduction_callback, checkpoint_callback]

  history = model.fit(
      train_images,
      train_labels,
      epochs=max_epochs,
      batch_size=batch_size,
      validation_split=0.2,
      callbacks=callbacks,
      verbose=1
      )
  return history

Застосуємо новостворені оптимізації на раніше збереженій моделі та виведемо результати:

In [ ]:
initial_model = keras.models.load_model('tdp1_model_v1.keras')

history = compile_and_fit(initial_model)
display_results(history)

Як можемо побачити, хоча запровадження зменшення швидкості навчання збільшило кількість епох з 10 до приблизно 25, результативні точність та помилка на тестових та валідаційних даних кращі.

Спробуємо тепер різні за розміром архітектури (кількість шарів та нейронів) з новоствореними пасивними оптимізаціями, аби визначити найбільш оптимальний розмір моделі. Визначимо початкові архітектури:

In [ ]:
model_tiny = keras.Sequential(
    [
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
    ],
    name='TDP1_tiny'
)

model_small = keras.Sequential(
    [
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
    ],
    name='TDP1_small'
)

model_medium = keras.Sequential(
    [
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
    ],
    name='TDP1_medium'
)

model_large = keras.Sequential(
    [
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
    ],
    name='TDP1_large'
)

Проведемо компіляцію та тренування цих моделей:

In [ ]:
size_histories = {}
size_histories['initial'] = history
size_histories['tiny'] = compile_and_fit(model_tiny)
size_histories['small'] = compile_and_fit(model_small)
size_histories['medium'] = compile_and_fit(model_medium)
size_histories['large'] = compile_and_fit(model_large)

Виведемо результати на 4 графіки для кожної з моделей:

In [ ]:
display_results(size_histories['tiny'])
display_results(size_histories['small'])
display_results(size_histories['medium'])
display_results(size_histories['large'])

Додатково, порівняємо на графіку результати помилки валідації всіх моделей, аби визначити найбільш підходящу для подальшої оптимізації архітектуру:

In [ ]:
def display_all_results(histories, value, miny=0.25, maxy=0.45, title='Порівняння', ylabel=None, epochs=30):
  plt.figure(figsize=(10, 6))
  for name, history in histories.items():
    plt.plot(history.history[value], label=name)

  plt.xlabel("Епохи")
  plt.ylabel(ylabel)
  plt.title(title)
  plt.legend()
  plt.grid(alpha=0.2)
  plt.xlim([0, epochs])
  plt.ylim([miny, maxy])
  plt.show()

In [ ]:
display_all_results(size_histories, 'val_loss', miny=0.3, maxy=0.5, title='Порівняння помилки валідації', ylabel='Помилка валідації')

Як можемо побачити, першопочаткова модель є занадто нестабільною через один прихований слой. Розглянемо інші моделі:


*   Крихітна модель - відбувається недонавчання, висока помилка на всіх епохах;
*   Мала модель - хороша помилка, однак ближче до 10 епох починаться перенавчання;
*   Середня модель - найнижча помилка протягом декількох епох, найбільш незначне перенавчання;
*   Велика модель - очевидне перенавчання;

Роблячи висновок із зображених на графіку результатів, найбільш підходящою моделлю для подальшого використання та оптимізації є середня модель. Мала модель є другим найкращим претендентом, і може буде використаною у ситуаціях де більш швидка модель є приорітетом точній моделі, але середня модель, в даному випадку, буде найбільш збалансованою, тож використаємо її.





Збережемо середню модель для подальших експериментів:

In [ ]:
model_medium.save('tdp1_model_v2.keras')
print('\nЗбережено модель tdp1_model_v2')

# **Ліквідація перенавчання моделі**

Спробуємо застосувати подальші оптимізації, такі як L1/L2 регуляризації та Dropout, аби забезпечити ліквідацію перенавчання і подальше збільшення точності результатів.

Створимо дві моделі, що базуються на щойно збереженій середній моделі, кожна з яких застосуватиме різні види регуляризації: L1, L2 та L1 + L2:

In [ ]:
model_medium_l1 = keras.Sequential(
    [
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l1(1e-5)),
    keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l1(1e-5)),
    keras.layers.Dense(10, activation='softmax')
    ],
    name='TDP1_medium_l1'
)

model_medium_l2 = keras.Sequential(
    [
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(1e-4)),
    keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(1e-4)),
    keras.layers.Dense(10, activation='softmax')
    ],
    name='TDP1_medium_l2'
)

model_medium_l1l2 = keras.Sequential(
    [
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l1_l2(l1=1e-5, l2=1e-4)),
    keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l1_l2(l1=1e-5, l2=1e-4)),
    keras.layers.Dense(10, activation='softmax')
    ],
    name='TDP1_medium_l1l2'
)

Виконаємо компіляцію та тестування нових моделей для порівняння результатів з оригіналом:

In [ ]:
initial_model = keras.models.load_model('tdp1_model_v2.keras')

optimization_histories = {}
optimization_histories['initial'] = size_histories['medium']
optimization_histories['l1'] = compile_and_fit(model_medium_l1)
optimization_histories['l2'] = compile_and_fit(model_medium_l2)
optimization_histories['l1l2'] = compile_and_fit(model_medium_l1l2)

Виведемо на 3 графіках результати для кожної з нових моделей:

In [ ]:
display_results(optimization_histories['l1'])
display_results(optimization_histories['l2'])
display_results(optimization_histories['l1l2'])

Виведемо результати фактичної помилки валідаційних даних всіх моделей для порівняння і визначення ефективності оптимізаційних методів, з урахуванням регуляризаційних штрафів:

In [ ]:
display_all_results(optimization_histories, 'val_xent', miny=0.28, title='Порівняння фактичної помилки валідації', ylabel='Фактична помилка валідації')

Для додаткової наглядності та остаточного рішення, виведемо точність валідаційних даних всіх моделей:

In [ ]:
display_all_results(optimization_histories, 'val_accuracy', miny=0.84, maxy=0.91, title='Порівняння точності валідації', ylabel='Точність валідації')

Як результат тестувань, встановимо 0.00001, як значення параметрeа L1 та 0.0001, як значення параметра L2, вони є найбільш ідеальними для конкретно цієї тестової вибірки.

У висновку, можемо побачити, що:


*   Без регуляризації: нестабільна точність та перенавчання даними;
*   Регуляризація L1: ліквідація перенавчання, однак гірші результати за відсутність регуляризації та гірша стабільність;
*   Регуляризація L2: ліквідація перенавчання, задовільні результати та задовільна стабільність;
*   Регуляризація L1 + L2: ліквідація перенавчання, кращі результати за відсутність регуляризації та найкраща стабільність.

Як результат, оберемо регуляризацію L1 + L2 і продовжимо оптимізацію.





Збережемо найкращу модель як основну:

In [ ]:
model_medium_l1l2.save('tdp1_model_v3.keras')
print('\nЗбережено модель tdp1_model_v3')

Додамо Dropout як фінальний метод оптимізації. Створимо три нові моделі, аби знайти найбільш оптимальне значення Dropout - моделі міститимуть слої Dropout зі значеннями 0.2, 0.35 і 0.5, відповідно:

In [ ]:
model_medium_l1l2_dropout02 = keras.Sequential(
    [
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l1_l2(l1=1e-5, l2=1e-4)),
    keras.layers.Dropout(0.20),
    keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l1_l2(l1=1e-5, l2=1e-4)),
    keras.layers.Dropout(0.20),
    keras.layers.Dense(10, activation='softmax')
    ],
    name='TDP1_medium_l1l2_dropout0.2'
)

model_medium_l1l2_dropout035 = keras.Sequential(
    [
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l1_l2(l1=1e-5, l2=1e-4)),
    keras.layers.Dropout(0.35),
    keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l1_l2(l1=1e-5, l2=1e-4)),
    keras.layers.Dropout(0.35),
    keras.layers.Dense(10, activation='softmax')
    ],
    name='TDP1_medium_l1l2_dropout0.35'
)

model_medium_l1l2_dropout05 = keras.Sequential(
    [
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l1_l2(l1=1e-5, l2=1e-4)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l1_l2(l1=1e-5, l2=1e-4)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
    ],
    name='TDP1_medium_l1l2_dropout0.5'
)

Виконаємо компіляцію та тестування нових моделей для визначення найкращого значення Dropout:

In [ ]:
initial_model = keras.models.load_model('tdp1_model_v3.keras')

dropout_histories = {}
dropout_histories['initial'] = optimization_histories['l1l2']
dropout_histories['d0.2'] = compile_and_fit(model_medium_l1l2_dropout02)
dropout_histories['d0.35'] = compile_and_fit(model_medium_l1l2_dropout035)
dropout_histories['d0.5'] = compile_and_fit(model_medium_l1l2_dropout05)

Виведемо на 3 різні графіки результати тестувань та валідації кожної з нових моделей:

In [ ]:
display_results(dropout_histories['d0.2'])
display_results(dropout_histories['d0.35'])
display_results(dropout_histories['d0.5'])

Виведемо результати фактичної помилки всіх моделей для порівняння ефективності різних значень Dropout на одному графіку:

In [ ]:
display_all_results(dropout_histories, 'val_xent', miny=0.28,
                    title='Порівняння фактичної помилки валідації', ylabel='Фактична помилка валідації',
                    epochs=75)

Можемо побачити, що найкращим серед обраних значень значенням Dropout є 0.2, тож застосуємо його і збережемо цю версію моделі:

In [ ]:
model_medium_l1l2_dropout02.save('tdp1_model_v4.keras')
print('\nЗбережено модель tdp1_model_v4')

Нарешті, налаштуємо розмір батчу шляхом порівняння різних значень параметра:

In [ ]:
initial_model = keras.models.load_model('tdp1_model_v4.keras')
model_b16 = initial_model
model_b32 = initial_model
model_b64 = initial_model
model_b128 = initial_model

batchsize_histories = {}
batchsize_histories['initial'] = dropout_histories['d0.2']
batchsize_histories['b16'] = compile_and_fit(model_b16, batch_size=16)
batchsize_histories['b32'] = compile_and_fit(model_b32, batch_size=32)
batchsize_histories['b64'] = compile_and_fit(model_b64, batch_size=64)
batchsize_histories['b128'] = compile_and_fit(model_b128, batch_size=128)

Виведемо результати валідаційних фактичної помилки та точності на два графіки:

In [ ]:
display_all_results(batchsize_histories, 'val_xent', miny=0.28,
                    title='Порівняння фактичної помилки валідації', ylabel='Фактична помилка валідації',
                    epochs=40)

In [ ]:
display_all_results(batchsize_histories, 'val_accuracy', miny=0.84, maxy=0.91,
                    title='Порівняння точності валідації', ylabel='Точність валідації',
                    epochs=40)

Як можемо побачити, розміри батчу в 16 і 32 мають значно кращі результати за розмір за замовчуванням, однак розмір батчу в 32 має кращу швидкість за нижчі розміри, тож оберемо саме його.

Так як архітектура моделі залишилася незмінною, збережемо лише нові гіперпараметри, окремо:

In [ ]:
model_b32.save_weights('tdp1_model_v4.weights.h5')
print('\nЗбережено вагові коефіцієнти моделі tdp1_model_v4')

Нарешті, протестуємо налаштовану та оптимізовану модель на реальних тестових даних:

In [ ]:
final_model = keras.Sequential(
    [
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l1_l2(l1=1e-5, l2=1e-4)),
    keras.layers.Dropout(0.20),
    keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l1_l2(l1=1e-5, l2=1e-4)),
    keras.layers.Dropout(0.20),
    keras.layers.Dense(10, activation='softmax')
    ]
)
final_model.load_weights('tdp1_model_v4.weights.h5')

In [ ]:
print("\n--- Перевіряємо точність фінальної моделі на тестових даних ---")

final_test_loss, final_test_accuracy = final_model.evaluate(test_images, test_labels, verbose=2)

print("\nФінальна тестова точність:", final_test_accuracy)
print("Фінальна тестова помилка:", final_test_loss)

In [ ]:
best_checkpoint_model = keras.models.load_model('best_model_checkpoint.keras')

print("\n--- Перевіряємо точність автоматично збереженої найкращої моделі на тестових даних ---")

checkpoint_test_loss, checkpoint_test_accuracy = best_checkpoint_model.evaluate(test_images, test_labels, verbose=2)

print("\nТочність найкращої моделі з чекпоінту:", checkpoint_test_accuracy)
print("Помилка найкращої моделі з чекпоінту:", checkpoint_test_loss)

In [ ]:
if final_test_accuracy >= checkpoint_test_accuracy:
    best_model_accuracy = final_test_accuracy
    best_model_loss = final_test_loss
    best_model_name = "фінальна модель"
else:
    best_model_accuracy = checkpoint_test_accuracy
    best_model_loss = checkpoint_test_loss
    best_model_name = "найкраща модель з чекпоінту"

print(f"\nВибрано {best_model_name} для порівняння з початковою моделлю.")

accuracy_improvement = ((best_model_accuracy - test_accuracy) / test_accuracy) * 100
loss_reduction = ((test_loss - best_model_loss) / test_loss) * 100

print("Покращення тестової точності: {:.2f}%".format(accuracy_improvement))
print("Покращення тестової помилки: {:.2f}%".format(loss_reduction))

Можемо побачити покращення результатів як точності, так і помилки моделі. Як наслідок, створено та оптимізовану просту модель для задачі класифікації рівномірно розподілених по класах зображень.